In [ ]:
#!pip install pypdf
#!pip install python-dotenv

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers

In [ ]:
!pip install -q datasets loralib sentencepiece

In [ ]:
!pip install -q einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index

In [ ]:
#pip install -U flash-attn --no-build-isolation

In [1]:

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#documents = SimpleDirectoryReader("example_data").load_data()

In [3]:
def get_meta(file_path):
    return {"file_name": file_path.split("/")[-1]}

reader = SimpleDirectoryReader(input_dir="/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts",recursive=True,
                              required_exts=[".py"])

documents = reader.load_data()

In [4]:
len(documents)

35

In [5]:
documents[0]

Document(id_='d837c43e-295f-473d-aba3-e3e04ab1782c', embedding=None, metadata={'file_path': '/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts/Hot_Peppers.py', 'file_name': 'Hot_Peppers.py', 'file_type': 'text/x-python', 'file_size': 20914, 'creation_date': '2023-12-20', 'last_modified_date': '2023-12-20', 'last_accessed_date': '2023-12-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='08523b607c9e3668532ce2f5ca673ebb63953a6501347159d0acb7b356d6efca', text='import os\nimport cv2\nimport collections\nimport numpy as np\nimport pandas as pd\nimport boto3\nimport warnings\nfrom tqdm import tqdm\nfrom statistics import median\nfrom skimage.morphology import medial_axis\nfrom scipy.interpolate import splprep, splev\nfrom scipy.ndimage import di

In [6]:

#!huggingface-cli login


In [7]:
from llama_index.prompts.prompts import SimpleInputPrompt
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "auto" # the device to load the model onto


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1000,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map=device,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float32},

)
     
#llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

#prompt = "My favourite condiment is"

#model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
#model.to(device)

#generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
#tokenizer.batch_decode(generated_ids)[0]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
embed_model =HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {'device': 'cpu'})

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en",model_kwargs = {'device': 'cpu'})

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en
Load pretrained SentenceTransformer: BAAI/bge-base-en


In [11]:
#!python -m pip install -U angle-emb

In [14]:
## SOTA Embeddings from huggingface leaderboard

from typing import Any, List
from llama_index.embeddings.base import BaseEmbedding
from llama_index.bridge.pydantic import PrivateAttr

## Embedding model specific current SOTA in huggung face leaderboard
from angle_emb import AnglE, Prompts


class UAEmbeddings(BaseEmbedding):
    
    _model: Any= PrivateAttr()
   
    def __init__(
        self,
        instructor_model_name: str = 'WhereIsAI/UAE-Large-V1',
        **kwargs: Any,
    ) -> None:
        
        super().__init__(**kwargs)  
        self._model=AnglE.from_pretrained(instructor_model_name, pooling_strategy='cls',device="cpu")
        #self._model.set_prompt(prompt=None)

        
    def _aget_query_embedding(self, query: str) -> List[float]:
        # Implementation of the abstract method
        embeddings = self._model.encode([ query])
        return embeddings[0]
        
    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([query])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([text])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode(
            [text for text in texts]
        )
        return embeddings

In [16]:
embed_model=UAEmbeddings()
embed_model._model.device
#embeddings = embed_model._get_text_embedding("It is raining cats and dogs here!")
#embeddings.shape

(1024,)

In [ ]:
#!pip install InstructorEmbedding


In [ ]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR

from llama_index.bridge.pydantic import PrivateAttr
from llama_index.embeddings.base import BaseEmbedding


class InstructorEmbeddings(BaseEmbedding):
    _model: INSTRUCTOR = PrivateAttr()
    _instruction: str = PrivateAttr()

    def __init__(
        self,
        instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent code base to easily search from:",
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name,device="cpu")
        self._instruction = instruction
        super().__init__(**kwargs)

    @classmethod
    def class_name(cls) -> str:
        return "instructor"

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, query]])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, text]])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode(
            [[self._instruction, text] for text in texts]
        )
        return embeddings

In [ ]:
embed_model=InstructorEmbeddings()
#embeddings = embed_model._get_text_embedding("It is raining cats and dogs here!")

In [ ]:
embeddings.shape

In [17]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [18]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [19]:
query_engine = index.as_query_engine()

In [20]:
response = query_engine.query("How is curved backbone of hot peppers calculated?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The curved backbone of hot peppers is calculated using the `get_curved_widths` function in the `phenotype_measurement_pepper` function. This function takes in the extended backbone of the hot pepper and the original image as input and returns the top and bottom of the curved backbone as well as the widths of the curved backbone. The function first calculates the widths of the curved backbone using the `cv2.arcLength` function and then finds the top and bottom of the curved backbone using the `np.where` function. The function then returns the top, bottom, and widths of the curved backbone.


In [21]:
response = query_engine.query("How to turn on/off visulaizations of phenotypes?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To turn on/off visualizations of phenotypes, you need to modify the "visualize\_phenotype" variable in the script. If "visualize\_phenotype" is set to True, the phenotype image will be saved and displayed. If it is set to False, the phenotype image will not be saved or displayed.

For example, in the script "Hot\_Peppers.py", the "visualize\_phenotype" variable is set as follows:
```python
visualize_phenotype = True
```
If you want to turn off the visualization of phenotypes, you can set "visualize\_phenotype" to False:
```python
visualize_phenotype = False
```
Similarly, in the script "multigpu/single\_hot\_peppers.py", the "visualize\_phenotype" variable is set as follows:
```python
visualize_phenotype = True
```
If you want to turn off the visualization of phenotypes, you can set "visualize\_phenotype" to False:


In [25]:
response = query_engine.query("How to turn on/off visulaizations of detections and phenotypes of hot peppers lsit all methods?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You can turn on/off visualizations of detections and phenotypes of hot peppers by setting the vis parameter to True or False respectively. The following methods are available to turn on/off visualizations:

* phenotype\_measurement\_pepper(img, patch, fname, fruitIdx, vis=False, ppx=None, ppy=None)
* run\_inference(det, learn, img, vis=False)

You can call these methods with the vis parameter set to True or False to turn on/off visualizations.
